In [1]:
import os
import awkward as ak
import numpy as np
import uproot
from coffea.nanoevents import NanoEventsFactory
from ecalphisym import EcalPhiSymSchema

In [2]:
import matplotlib.pyplot as plt
import mplhep
%matplotlib inline
plt.rc('font',**{'family':'sans-serif', 'sans-serif':['Helvetica'], 'size':'20'})
plt.rcParams['figure.figsize'] = '10,10'
plt.rcParams['xaxis.labellocation'] = 'right'
plt.rcParams['yaxis.labellocation'] = 'top'
plt.style.use(mplhep.style.ROOT)

In [3]:
# path to W/Z E/p - PhiSym 
runs = NanoEventsFactory.from_root('../Run2018D_test.root',
                                   schemaclass=EcalPhiSymSchema,
                                   treepath="/Runs").events()
eop = uproot.open('../EGamma-Run2018B-ZSkim-17Sep2018-v1-316998-319312.root:selected')

In [4]:
counts = np.unique(runs.EcalPhiSymInfo.fill, return_index=True)[1]
splits = np.diff(np.concatenate([counts, [len(runs.EcalPhiSymInfo.fill)]]))
ebhits = ak.unflatten(runs.EcalPhiSymEB, splits, axis=0, behavior=runs.behavior).sum(axis=1)

In [5]:
ak_eop = eop.arrays(filter_name = "/charge|eta/")

In [6]:
# ieat histo phisym
hist_phisym,bins =np.histogram(ak.to_numpy(ebhits.ieta[1,:]), weights=ak.to_numpy(ebhits.sumet[1,:]), 
           bins=171, range=[-85.5, 85.5])
hist_phisym = np.delete(hist_phisym, 85)


In [7]:
# ieta histo eop
hist_eop,bins =np.histogram(ak.to_numpy(ak.mask(ak_eop.etaEle[:,0], (ak_eop.chargeEle[:,0]  * ak_eop.chargeEle[:,1]  == 0)  , valid_when=False))/0.0175, bins=171, range=[-85.5, 85.5])
hist_eop = np.delete(hist_eop, 85) 


In [8]:
weights = hist_eop/hist_phisym * (np.sum(hist_phisym) / np.sum(hist_eop) )

In [9]:
# save weights ordered from -85 to 85
np.savetxt('weight.txt', weights, delimiter=',')   
